In [2]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

default checking
Running command `conda list`... ok
jupyterlab=1.2.6 already installed
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
psutil=5.7.0 already installed
scipy=1.4.1 already installed
statsmodels=0.11.0 already installed
scikit-learn=0.22.1 already installed
xlrd=1.2.0 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Naive Bayes

## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Problema

Vamos a construir un clasificador que intente predecir si una persona ganará más de US$ 50.000 por año 

Hicimos preparación de los datos en la práctica de checkpoint. Los datasets resultado de esa práctica son el input de ésta. Si no la hicieron, comiencen con esa y después sigan en esta notebook.


## Dataset

https://archive.ics.uci.edu/ml/datasets/Adult

Los datos corresponden a un censo de 1994.

Los campos son

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-

op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, 
Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


In [4]:
data_train_location = '../Data/adult_train.csv'
data_test_location = '../Data/adult_test.csv'

data_train = pd.read_csv(data_train_location, sep='\t', low_memory=False)
data_test = pd.read_csv(data_test_location, sep='\t', low_memory=False)

data_train.head(3)

,workclass_State-gov,workclass_Self-emp-not-inc,workclass_Private,workclass_Federal-gov,workclass_Local-gov,workclass_Self-emp-inc,workclass_Without-pay,education_Bachelors,education_HS-grad,education_11th,...,native_country_Ireland,native_country_Hungary,native_country_Holand-Netherlands,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1.000000,-1.111697,0.00,0.0,0.0,-5.0,<=50K
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,-0.736842,1.477544,-0.25,0.0,0.0,0.0,<=50K
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.157895,0.436143,-0.25,0.0,625.0,-0.4,<=50K


In [5]:
data_test.head(3)

,workclass_State-gov,workclass_Self-emp-not-inc,workclass_Private,workclass_Federal-gov,workclass_Local-gov,workclass_Self-emp-inc,workclass_Without-pay,education_Bachelors,education_HS-grad,education_11th,...,native_country_Ireland,native_country_Hungary,native_country_Holand-Netherlands,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.736842,0.438467,0.00,0.0,0.0,0.0,<=50K
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.057352,0.00,0.0,0.0,3.0,<=50K
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,-0.368421,-0.801619,-0.25,0.0,0.0,0.0,<=50K


## Ejercicio 1 - Freatures Target

Sabiendo que 'income' es el nombre de la columna target, construyamos la matriz de features y el vector target para los conjuntos de train y test

## Ejercicio 2 - Training

Instanciemos y entrenemos uno modelo naive bayes gaussiano.

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

## Ejercicio 3 - Predict

Usemos el modelo entrenado en el ejercicio 2 para predecir la etiqueta de los datos de test

## Ejercicio 4 - Performance

Para los datos de test, calculemos accuracy:

Accuracy = (TP + TN)/(TP + TN + FP + FN)

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

Comparemos la performance que obtuvimos con la performance del modelo nulo.

El modelo nulo es el que predice todas las instancias con la etiqueta de la clase mayoritaria.

¿Cuál es la clase mayoritaria?

La clase mayoritaria es <=50K.

Calculemos null_acuracy como si en test hubieramos predicho <=50K para todos los registros:

Calculemos la matriz de confusión sobre los datos de test y grafiquemos el heatmap de esta matriz.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html